In [ ]:
import numpy as np
import seaborn as sns
import hydra
import torch
import itertools
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from omegaconf import OmegaConf
from pytorch_lightning import (
    LightningDataModule,
    LightningModule,
    seed_everything,
)

from multimodal_contrastive.analysis.utils import *
from multimodal_contrastive.utils import utils

In [ ]:
sns.set_theme(style="whitegrid")

In [ ]:
# register custom resolvers if not already registered
OmegaConf.register_new_resolver("sum", lambda input_list: np.sum(input_list), replace=True)

device = "cuda" if torch.cuda.is_available() else "cpu"
device

In [ ]:
# Load config for CLIP model
config_name = "puma_sm_gmc"
configs_path = "../../configs"

with hydra.initialize(version_base=None, config_path=configs_path):
    cfg = hydra.compose(config_name=config_name)

cfg.datamodule.split_type

In [ ]:
# set seed for random number generators in pytorch, numpy and python.random
# and especially for generating the same data splits for the test set
if cfg.get("seed"):
    seed_everything(cfg.seed, workers=True)

In [ ]:
# Load test data split
datamodule: LightningDataModule = hydra.utils.instantiate(cfg.datamodule)
datamodule.setup("test")

# train_loader = datamodule.train_dataloader()
test_loader = datamodule.infer_dataloader()
val_loader = datamodule.val_dataloader()
train_idx, val_idx, test_idx = datamodule.get_split_idx()

In [ ]:
# Load model from checkpoint
ckpt_path = "path.to/gmc_proxy.ckpt"
model: LightningModule = utils.instantiate_model(cfg)
model = model.load_from_checkpoint(ckpt_path, map_location=device)
model = model.eval()

In [ ]:
# Set which loader to use
# loader = test_loader
loader, filtered_ge_idx = test_loader, test_idx
loader = make_eval_data_loader(loader.dataset)

# Peek at the first example in the loaded set
example = next(iter(loader))
inputs = example["inputs"]
struct, morph = inputs["struct"], inputs["morph"]

print(struct)
print(morph.shape)

In [ ]:
# Get latent representations for all test examples
representations, mols = model.compute_representation_dataloader(
    loader,
    device=device,
    return_mol=True
)

In [ ]:
# If dataset is too large, randomly subset to cutoff molecules
p_mols, p_representations, p_idx = random_subset(mols, representations, cutoff=3000)
p_representations["morph"].shape

In [ ]:
# Use rdkit to get Morgan fingerprints for each molecule in mols
mol_fps = get_molecular_fingerprints(p_mols, fp_type="morgan", nbits=2048)
tanimoto_sim = get_molecular_similarity(mol_fps, "tanimoto")

In [ ]:
# Get the raw domain data (gene expression & cell profiler features) from the dataloader
mods = unroll_dataloader(loader, mods=['morph'])
# ge = mods['ge'][p_idx]
morph = mods['morph'][p_idx]

In [ ]:
# run pca on raw gene expression matrix
N_genes=30
pca = PCA(n_components=N_genes)
ge_pca = pca.fit_transform(ge)
ge_pca.shape

In [ ]:
# run pca on raw morphology (Cell Profiler) features
N_morph_f = 30
pca = PCA(n_components=N_morph_f)
morph_pca = pca.fit_transform(morph)
morph_pca.shape

In [ ]:
# Subset raw gene expression to top-k=81 most differentially expressed genes
filtered_ge = np.load("adata_scale_filt.npy")
filtered_ge = filtered_ge[filtered_ge_idx][p_idx]
filtered_ge.shape

In [ ]:
# Single modality latent distance versus tanimoto similarity
mods_x = ['struct', 'morph', 'joint']
mods_y = tanimoto_sim

fig, ax = plt.subplots(2, 3, figsize=(15, 10))

for idx, mod_x in enumerate(mods_x):
    x = get_pairwise_similarity(
        p_representations[mod_x],
        p_representations[mod_x],
        metric="cosine"
    )
    x, y = get_values_from_dist_mat(x, mods_y, keep_diag=False)
    corr = np.corrcoef(x, y)
    mod_x_label = mod_x.capitalize()

    # Sample a subset of the data for visualization
    rnd_idx = np.random.choice(len(x), 15000, replace=False)
    x, y = x[rnd_idx], y[rnd_idx]

    ax[0, idx].scatter(x, y, alpha=1, s=1)
    ax[0, idx].set_xlabel(f"Cosine Similarity ({mod_x}-{mod_x})")
    ax[0, idx].set_ylabel("Tanimoto Similarity")
    ax[0, idx].set_title(f"Latent {mod_x_label} Cosine Sim. vs Tanimoto Sim.")
    ax[0, idx].legend(title=f"$r={corr[0, 1]:.2f}$", loc="upper left")

for idx, mod_x in enumerate(mods_x):
    x = get_pairwise_similarity(
        p_representations[mod_x],
        p_representations[mod_x],
        metric="euclidean"
    )
    x, y = get_values_from_dist_mat(x, mods_y, keep_diag=False)
    corr = np.corrcoef(x, y)
    mod_x_label = mod_x.capitalize()

    # Sample a subset of the data for visualization
    rnd_idx = np.random.choice(len(x), 15000, replace=False)
    x, y = x[rnd_idx], y[rnd_idx]
    
    ax[1, idx].scatter(x, y, alpha=1, s=1)
    ax[1, idx].set_xlabel(f"Euclidean Distance ({mod_x}-{mod_x})")
    ax[1, idx].set_ylabel("Tanimoto Similarity")
    ax[1, idx].set_title(f"Latent {mod_x_label} Euclidean Dist. vs Tanimoto Sim.")
    ax[1, idx].legend(title=f"$r={corr[0, 1]:.2f}$", loc="upper right")

plt.tight_layout()
plt.savefig("single_modality_vs_tanimoto.pdf", dpi=300, format="pdf")
plt.show()

In [ ]:
# plotting different modalities of the same samples against each other
mods = ['struct', 'morph', 'joint']

fig, ax = plt.subplots(1, 3, figsize=(15, 5), squeeze=False)
ctr = 0

for idx in range(len(mods)):
    for idy in range(idx+1, len(mods)):
        x = get_pairwise_similarity(
            p_representations[mods[idx]],
            p_representations[mods[idx]],
            metric="cosine"
        )
        y = get_pairwise_similarity(
            p_representations[mods[idy]],
            p_representations[mods[idy]],
            metric="cosine"
        )
        x, y = get_values_from_dist_mat(x, y, keep_diag=False)
        corr = np.corrcoef(x, y)

        # Sample a subset of the data for visualization
        rnd_idx = np.random.choice(len(x), 15000, replace=False)
        x, y = x[rnd_idx], y[rnd_idx]
        
        ax_idx = 0

        ax[ax_idx, ctr%3].scatter(x, y, alpha=1, s=1)
        ax[ax_idx, ctr%3].set_xlabel(f"Cosine Similarity ({mods[idx]}-{mods[idx]})")
        ax[ax_idx, ctr%3].set_ylabel(f"Cosine Similarity ({mods[idy]}-{mods[idy]})")
        ax[ax_idx, ctr%3].set_title(f"Latent {mods[idx].capitalize()} vs {mods[idy].capitalize()} Cosine Similarity")
        ax[ax_idx, ctr%3].legend(title=f"$r={corr[0, 1]:.2f}$", loc="upper left")
        ctr += 1
        
plt.tight_layout()
plt.savefig("single_modality_vs_single_modality.pdf", dpi=300, format="pdf")
plt.show()

In [ ]:
# plot cross-modality latent similarity versus raw data domain similarity (gene expression & Tanimoto sim)
mods_x = ['struct', 'ge', 'joint']
ge_sim = get_pairwise_similarity(ge, ge, metric="cosine")

fig, ax = plt.subplots(3, 3, figsize=(15, 12))

for idx, (mod_a, mod_b) in enumerate(itertools.combinations(mods_x, 2)):
    x = get_pairwise_similarity(
        p_representations[mod_a],
        p_representations[mod_b],
        metric="cosine"
    )
    x, y = get_values_from_dist_mat(x, tanimoto_sim, keep_diag=True)
    corr = np.corrcoef(x, y)

    ax[0, idx].scatter(x, y, alpha=0.5, s=.1)
    ax[0, idx].set_xlabel(f"Cosine Similarity ({mod_a}-{mod_b})")
    ax[0, idx].set_ylabel("Tanimoto Similarity")
    ax[0, idx].set_title(f"({mod_a}-{mod_b}) vs Tanimoto (r={corr[0, 1]:.2f})")
    
for idx, (mod_a, mod_b) in enumerate(itertools.combinations(mods_x, 2)):
    x = get_pairwise_similarity(
        p_representations[mod_a],
        p_representations[mod_b],
        metric="cosine"
    )
    y = ge_sim
    x, y = get_values_from_dist_mat(x, y, keep_diag=True)
    corr = np.corrcoef(x, y)

    ax[1, idx].scatter(x, y, alpha=0.5, s=.1)
    ax[1, idx].set_xlabel(f"Cosine Similarity ({mod_a}-{mod_b})")
    ax[1, idx].set_ylabel("Cosine Similarity (raw gene exp)")
    ax[1, idx].set_title(f"({mod_a}-{mod_b}) vs raw GE (r={corr[0, 1]:.2f})")
    
for idx, (mod_a, mod_b) in enumerate(itertools.combinations(mods_x, 2)):
    x = get_pairwise_similarity(
        p_representations[mod_a],
        p_representations[mod_b],
        metric="cosine"
    )
    y = ge_sim
    x, y = get_values_from_dist_mat(x, y, keep_diag=True)
    corr = np.corrcoef(x, y)

    ax[2, idx].scatter(x, y, alpha=0.5, s=.1)
    ax[2, idx].set_xlabel(f"Cosine Similarity ({mod_a}-{mod_b})")
    ax[2, idx].set_ylabel("Euclidean Distance (raw gene exp)")
    ax[2, idx].set_title(f"({mod_a}-{mod_b}) vs raw GE (r={corr[0, 1]:.2f})")
        
plt.tight_layout()
plt.savefig("cross_modality_vs_raw_sim.png")
plt.show()

In [ ]:
# plot cross-modality latent similarity versus PCA morph data
mods_x = ['struct', 'morph', 'joint']
morph_pca_sim_cos = get_pairwise_similarity(morph_pca, morph_pca, metric="cosine")
morph_pca_sim_euc = get_pairwise_similarity(morph_pca, morph_pca, metric="euclidean")

fig, ax = plt.subplots(2, 3, figsize=(15, 10))

for idx, (mod_a, mod_b) in enumerate(itertools.combinations(mods_x, 2)):
    x = get_pairwise_similarity(
        p_representations[mod_a],
        p_representations[mod_b],
        metric="cosine"
    )
    x, y = get_values_from_dist_mat(x, tanimoto_sim, keep_diag=True)
    corr = np.corrcoef(x, y)

    # Sample a subset of the data for visualization
    rnd_idx = np.random.choice(len(x), 15000, replace=False)
    x, y = x[rnd_idx], y[rnd_idx]

    ax[0, idx].scatter(x, y, alpha=1, s=1)
    ax[0, idx].set_xlabel(f"Cosine Similarity ({mod_a}-{mod_b})")
    ax[0, idx].set_ylabel("Tanimoto Similarity")
    ax[0, idx].set_title(f"Cross-Modal Latent Cosine Sim. ({mod_a}-{mod_b})\nvs Tanimoto Sim.")
    ax[0, idx].legend(title=f"$r={corr[0, 1]:.2f}$", loc="upper left")
    
for idx, (mod_a, mod_b) in enumerate(itertools.combinations(mods_x, 2)):
    x = get_pairwise_similarity(
        p_representations[mod_a],
        p_representations[mod_b],
        metric="cosine"
    )
    y = morph_pca_sim_cos
    x, y = get_values_from_dist_mat(x, y, keep_diag=True)
    corr = np.corrcoef(x, y)

    # Sample a subset of the data for visualization
    rnd_idx = np.random.choice(len(x), 15000, replace=False)
    x, y = x[rnd_idx], y[rnd_idx]

    ax[1, idx].scatter(x, y, alpha=1, s=1)
    ax[1, idx].set_xlabel(f"Cosine Similarity ({mod_a}-{mod_b})")
    ax[1, idx].set_ylabel("Cosine Similarity (morph-morph PCA)")
    ax[1, idx].set_title(f"Cross-Modal Latent Cosine Sim. ({mod_a}-{mod_b})\nvs Morph PCA Cosine Sim.")
    ax[1, idx].legend(title=f"$r={corr[0, 1]:.2f}$")

plt.tight_layout()
plt.savefig("cross_modality_vs_pca_morph.pdf", dpi=300, format="pdf")
plt.show()

In [ ]:
morph_pca_sim_cos = get_pairwise_similarity(morph_pca, morph_pca, metric="cosine")
morph_pca_sim_euc = get_pairwise_similarity(morph_pca, morph_pca, metric="euclidean")

fig, ax = plt.subplots(1, 2, figsize=(10, 5))

for idx, (mod_a, mod_b) in enumerate([('struct', 'morph'), ('struct', 'joint')]):
    x = get_pairwise_similarity(
        p_representations[mod_a],
        p_representations[mod_b],
        metric="cosine"
    )
    y = morph_pca_sim_cos
    x, y = get_values_from_dist_mat(x, y, keep_diag=True)
    corr = np.corrcoef(x, y)

    # Keep only a subset of the data for visualization
    rnd_idx = np.random.choice(len(x), 15000, replace=False)
    x, y = x[rnd_idx], y[rnd_idx]

    ax[idx].scatter(x, y, alpha=1, s=1)
    ax[idx].set_xlabel(f"Cosine Similarity ({mod_a}-{mod_b})")
    ax[idx].set_ylabel("Cosine Similarity (morph-morph PCA)")
    ax[idx].set_title(f"Cross-Modal Latent Cosine Sim. ({mod_a.capitalize()}-{mod_b.capitalize()})\nvs Morph PCA Cosine Sim.")
    ax[idx].legend(title=f"$r={corr[0, 1]:.2f}$", loc="upper left")

plt.tight_layout()
plt.savefig("struct&joint_vs_pca_morph.pdf", dpi=300, format="pdf")
plt.show()

In [ ]:
# plot cross-modality latent similarity versus raw morph domain distance
mods_x = ['struct', 'morph', 'joint']
morph_sim_cos = get_pairwise_similarity(morph, morph, metric="cosine")
morph_sim_euc = get_pairwise_similarity(morph, morph, metric="euclidean")

fig, ax = plt.subplots(2, 3, figsize=(15, 8))

for idx, (mod_a, mod_b) in enumerate(itertools.combinations(mods_x, 2)):
    x = get_pairwise_similarity(
        p_representations[mod_a],
        p_representations[mod_b],
        metric="cosine"
    )
    y = morph_sim_cos
    x, y = get_values_from_dist_mat(x, y, keep_diag=True)
    corr = np.corrcoef(x, y)

    ax[0, idx].scatter(x, y, alpha=0.5, s=.1)
    ax[0, idx].set_xlabel(f"Cosine Similarity ({mod_a}-{mod_b})")
    ax[0, idx].set_ylabel(f"Cosine Similarity (raw morph features)")
    ax[0, idx].set_title(f"({mod_a}-{mod_b}) vs raw morph (r={corr[0, 1]:.2f})")
    
for idx, (mod_a, mod_b) in enumerate(itertools.combinations(mods_x, 2)):
    x = get_pairwise_similarity(
        p_representations[mod_a],
        p_representations[mod_b],
        metric="cosine"
    )
    y = morph_sim_euc
    x, y = get_values_from_dist_mat(x, y, keep_diag=True)
    corr = np.corrcoef(x, y)

    ax[1, idx].scatter(x, y, alpha=0.5, s=.1)
    ax[1, idx].set_xlabel(f"Cosine Similarity ({mod_a}-{mod_b})")
    ax[1, idx].set_ylabel(f"Euclidean Distance (raw morph features)")
    ax[1, idx].set_title(f"({mod_a}-{mod_b}) vs raw morph (r={corr[0, 1]:.2f})")
        
plt.tight_layout()
plt.savefig("cross_modality_vs_raw_morph.png")
plt.show()

In [ ]:
# plot cross-modality latent similarity versus PCA gene expression
mods_x = ['struct', 'ge', 'joint']
ge_pca_sim_cos = get_pairwise_similarity(ge_pca, ge_pca, metric="cosine")
ge_pca_sim_euc = get_pairwise_similarity(ge_pca, ge_pca, metric="euclidean")

fig, ax = plt.subplots(2, 3, figsize=(15, 8))
    
for idx, (mod_a, mod_b) in enumerate(itertools.combinations(mods_x, 2)):
    x = get_pairwise_similarity(
        p_representations[mod_a],
        p_representations[mod_b],
        metric="cosine"
    )
    y = ge_pca_sim_cos
    x, y = get_values_from_dist_mat(x, y, keep_diag=True)
    corr = np.corrcoef(x, y)

    ax[0, idx].scatter(x, y, alpha=0.5, s=.1)
    ax[0, idx].set_xlabel(f"Cosine Similarity ({mod_a}-{mod_b})")
    ax[0, idx].set_ylabel(f"Cosine Similarity (PCA gene exp, n={N_genes})")
    ax[0, idx].set_title(f"({mod_a}-{mod_b}) vs pca GE (r={corr[0, 1]:.2f})")
    
for idx, (mod_a, mod_b) in enumerate(itertools.combinations(mods_x, 2)):
    x = get_pairwise_similarity(
        p_representations[mod_a],
        p_representations[mod_b],
        metric="cosine"
    )
    y = ge_pca_sim_euc
    x, y = get_values_from_dist_mat(x, y, keep_diag=True)
    corr = np.corrcoef(x, y)

    ax[1, idx].scatter(x, y, alpha=0.5, s=.1)
    ax[1, idx].set_xlabel(f"Cosine Similarity ({mod_a}-{mod_b})")
    ax[1, idx].set_ylabel(f"Euclidean Distance (PCA gene exp, n={N_genes})")
    ax[1, idx].set_title(f"({mod_a}-{mod_b}) vs pca GE (r={corr[0, 1]:.2f})")
        
plt.tight_layout()
plt.savefig("cross_modality_vs_pca_ge.png")
plt.show()

In [ ]:
# Run same analysis as above, but with the top-k=81 most differentially expressed genes obtained from file
mods_x = ['struct', 'ge', 'joint']
filtered_ge_sim_cos = get_pairwise_similarity(filtered_ge, filtered_ge, metric="cosine")
filtered_ge_sim_euc = get_pairwise_similarity(filtered_ge, filtered_ge, metric="euclidean")
# filtered_ge_sim_mse = get_pairwise_similarity(filtered_ge, filtered_ge, metric="mse")

fig, ax = plt.subplots(2, 3, figsize=(15, 8))
    
for idx, (mod_a, mod_b) in enumerate(itertools.combinations(mods_x, 2)):
    x = get_pairwise_similarity(
        p_representations[mod_a],
        p_representations[mod_b],
        metric="cosine"
    )
    y = filtered_ge_sim_cos
    x, y = get_values_from_dist_mat(x, y, keep_diag=True)
    corr = np.corrcoef(x, y)

    ax[0, idx].scatter(x, y, alpha=0.5, s=.1)
    ax[0, idx].set_xlabel(f"Cosine Similarity ({mod_a}-{mod_b})")
    ax[0, idx].set_ylabel(f"Cosine Similarity (DE gene exp, n=81)")
    ax[0, idx].set_title(f"({mod_a}-{mod_b}) vs filtered GE (r={corr[0, 1]:.2f})")
    
for idx, (mod_a, mod_b) in enumerate(itertools.combinations(mods_x, 2)):
    x = get_pairwise_similarity(
        p_representations[mod_a],
        p_representations[mod_b],
        metric="cosine"
    )
    y = filtered_ge_sim_euc
    x, y = get_values_from_dist_mat(x, y, keep_diag=True)
    corr = np.corrcoef(x, y)

    ax[1, idx].scatter(x, y, alpha=0.5, s=.1)
    ax[1, idx].set_xlabel(f"Cosine Similarity ({mod_a}-{mod_b})")
    ax[1, idx].set_ylabel(f"Euclidean Distance (DE gene exp, n=81)")
    ax[1, idx].set_title(f"({mod_a}-{mod_b}) vs filtered GE (r={corr[0, 1]:.2f})")
        
plt.tight_layout()
plt.savefig("cross_modality_vs_filtered_ge_top_81.png")
plt.show()

## Sanity Checks

In [ ]:
from sklearn import preprocessing

In [ ]:
filtered_ge = np.load("adata_scale_filt.npy")
scaled_filtered_ge = preprocessing.normalize(filtered_ge, axis=1)
filtered_test_ge = filtered_ge[test_idx]

In [ ]:
loader = make_eval_data_loader(datamodule.dataset)
mods = unroll_dataloader(loader, mods=['ge', 'morph'])
ge = mods['ge']

In [ ]:
# Check that there is overlap in the gene expression columns between the DE gene subset and the original
print(filtered_ge.shape, ge.shape)

de_idx = []

for col_1 in tqdm(filtered_ge.T):
    for idx, col_2 in enumerate(ge.T):
        if np.array_equal(col_1, col_2):
            de_idx.append(idx)
            
len(de_idx)

In [ ]:
# Compute mean and std deviation of the raw gene expression
u = np.mean(ge)
v = np.std(ge)

nu = np.mean(filtered_ge)
nv = np.std(filtered_ge)

snu = np.mean(scaled_filtered_ge)
snv = np.std(scaled_filtered_ge)

print(u, nu)
print(v, nv)

In [ ]:
print(np.linalg.norm(ge[0]))
print(np.linalg.norm(filtered_ge[0]))
print(np.linalg.norm(scaled_filtered_ge[0]))

In [ ]:
# Plot histogram of pairwise distances in the raw gene expression space versus the filtered gene expression space
ge_dist = get_pairwise_similarity(ge, ge, metric="euclidean")
filtered_ge_dist = get_pairwise_similarity(filtered_ge, filtered_ge, metric="euclidean")

In [ ]:
x, y = get_values_from_dist_mat(ge_dist, filtered_ge_dist)
x.shape, y.shape

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 3))

ax[0].hist(x, density=True, bins=40)
ax[0].set_ylabel('Density')
ax[0].set_xlabel('Eucl. dist. between pairs')
ax[0].set_title('Pairwise dist. in raw gene expr. data')

ax[1].hist(y, density=True, bins=40)
ax[1].set_ylabel('Density')
ax[1].set_xlabel('Eucl. dist. between pairs')
ax[1].set_title('Pairwise dist. in filtered gene expr. data')

plt.tight_layout()
plt.show()

In [ ]:
plt.plot(np.sort(ge[0]))
plt.show()

In [ ]:
plt.plot(np.sort(ge[1]))
plt.show()